# ECal Tester
This bit of code allows you to visualize on an event by event basis the contents of ECal MC hits. Different style of dots are super imposed
on the face of the ECal indicating the cluster centers of hits in the score plane in front of the ECal, the hits in that score plane, and
the hits in the ECal.

In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
from array import array
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
# R.EnableImplicitMT()

Warning in <TInterpreter::ReadRootmapFile>: While processing /data/HPS/lib/libBaseAna2.rootmap, class MCParticle was found to be associated to libBaseAna2.dylib although it is already associated to libevent.dylib


In [2]:
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Recompile is not needed


'Utility Functions V1.0.6 \n'

In [3]:
ret = R.gSystem.Load("/System/Volumes/Data/data/HPS/Analyses/build/ECAL/lib/libEcal_Analysis.dylib")
if(ret != 0):
    print("The return code was: ",ret)
else:
    R.gInterpreter.ProcessLine('''auto EAC = Ecal_Analysis_Class();''')   # This is key. It puts the EAC in C++ space.
print(f"{R.EAC.Version()}")
R.gSystem.Load("/data/HPS/lib/libMiniDST")  # Weirdest thing, if I do this first the EAC will be version 1.0.9 instead of 1.1.1

V1.1.2


0

In [4]:
ch = R.TChain("MiniDST")
# ch.Add("/data/HPS/data/MC/emumu_calchep/events_4p55GeV_000_slic_recon.root")
# ch.Add("/data/HPS/data/MC/mum_2019/mum_1_2019.root")
# ch.Add("/data/HPS/data/physrun2021/sim_2021/hpsForward_e-_3.0GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/sim_2021/new_e-*.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
ch.Add("/data/HPS/data/physrun2021/Tritrig/Pure/*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")

Number of events loaded:   2.093M


In [5]:
df = R.RDataFrame(ch)
try:
    dfx = R.EAC.extend_dataframe(R.RDF.AsRNode(df))
except:
    print("Wow, no extension")
    dfx = df
print("Available data names in Tuple:")
ColumnNames=dfx.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_mc_id            ecal_cluster_mc_pdg           
ecal_cluster_mc_pdg_purity    ecal_cluster_nhits            ecal_cluster_seed_energy      ecal_cluster_seed_index       
ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             ecal_cluster_uncor_energy     
ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energyecal_cluster_uncor_seed_index 
ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       ecal_cluster_uncor_x          
ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                ecal_cluster_y                
ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              ecal_hit_index_y              
ecal_hit_mc_contrib_ec        ecal_hit_mc_contrib_id        ecal_hit_mc_contrib_pdg       ecal_hit_mc_parent_id         


Ecal_Analysis_Class::extend_dataframe: The input dataframe does not have the required columns for this extension.


In [6]:
def make_ecal_snaphot(mini_dst, hist, opt=0):
    """Return a 2D histogram for the ECal, with the energy of the hits on the z-axis"""
    if hist is None:
        hist = R.TH2D("hist_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5)
    else:
        hist.Reset()
    # Fill the histogram by looping over the ECal hits
    for i in range(len(mini_dst.ecal_hit_index_x)):
        hist.Fill(mini_dst.ecal_hit_index_x[i], mini_dst.ecal_hit_index_y[i], mini_dst.ecal_hit_energy[i])
    return hist

# Setup the MiniDST class, which makes for easier event by event data inplection.
# This MiniDST class looks directly into the TTree, so does not use the RDataframe class.
# You need to:
mdst = R.MiniDst()          # Initiate the class
mdst.use_mc_particles=True  # Tell it to look for the MC Particles in the TTree
mdst.use_ecal_cluster_uncor = True
mdst.use_mc_scoring =True
mdst.DefineBranchMap()      # Define the map of all the branches to the contents of the TTree
mdst.SetBranchAddressesOnTree(ch) # Connect the TChain (which contains the TTree) to the class.
print(f"MiniDST version = {mdst._version_()}")
event = 1

MiniDST version = 1.2.2


In [7]:
cc2 = R.TCanvas("cc2","Canvas",900,600)
legend2 = None
hh = None
ones = None
dot_graph = None
dot2_graph = None
dot3_graph = None
dot4_graph = None
clus_dot_graph = None
clus_dot_uncor_graph = None
track_dot_graph = None
cl_idx = None


def Show_Event(debug=False):
    global hh, ones, dot_graph, dot2_graph, dot3_graph, clus_dot_graph, clus_dot_uncor_graph, track_dot_graph, legend2
    global cl_idx
    global dot4_graph

    hh = make_ecal_snaphot(mdst, hh, 0)
    hh.SetStats(0)
    hh.SetTitle(f"Ecal Hits event={event}")
    cc2.Clear()
    ones = fancy_plot(hh, ones, 0x0)
    pzsum = 0;
    if debug:
        print(f"Primary particle into scoring plane, index = {primary_index}")
    x_ave = 0
    y_ave = 0
    x_dots = array('d')
    y_dots = array('d')
    x_dots2 = array('d')
    y_dots2 = array('d')
    x_dots3 = array('d')
    y_dots3 = array('d')
    x_dots4 = array('d')
    y_dots4 = array('d')

    for i in range(len(mdst.mc_score_pdg)):
        if mdst.mc_score_z[i] > 1400 and mdst.mc_score_pz[i] > 0.01 and mdst.mc_score_part_idx[i] == primary_index :  # and mdst.mc_score_pz[i] > 2.8
            pzsum += mdst.mc_score_pz[i]
            x_ave += mdst.mc_score_x[i]*mdst.mc_score_pz[i]
            y_ave += mdst.mc_score_y[i]*mdst.mc_score_pz[i]
            if debug:
                print(f"[{mdst.mc_score_part_idx[i]:3d}] {mdst.mc_score_pdg[i]:3d} ({mdst.mc_score_x[i]:7.1f},{mdst.mc_score_y[i]:7.1f}) pz={mdst.mc_score_pz[i]:7.5f}")

    if pzsum == 0:
        x_ave = 0
        y_ave = 0
    else:
        x_ave = x_ave/pzsum
        y_ave = y_ave/pzsum
        x_dots.append(R.EAC.ecal_xpos_to_index(x_ave))
        y_dots.append(R.EAC.ecal_ypos_to_index(y_ave))

    if debug:
        print(f"Primary hits: (x,y)_ave = ({x_ave:7.1f}, {y_ave:7.1f}) Pz sum = {pzsum:7.4f}")
        print("Secondary particles:")
        for i in range(len(mdst.mc_score_pdg)):
            if mdst.mc_score_z[i] > 1400 and mdst.mc_score_pz[i] > 0.01 and mdst.mc_score_part_idx[i] != primary_index :  # and mdst.mc_score_pz[i] > 2.8
                x_dots2.append(R.EAC.ecal_xpos_to_index(mdst.mc_score_x[i]))
                y_dots2.append(R.EAC.ecal_ypos_to_index(mdst.mc_score_y[i]))
                pzsum += mdst.mc_score_pz[i]
                print(f"{len(x_dots2)-1:2d}:{i:3d} [{mdst.mc_score_part_idx[i]:3d}] {mdst.mc_score_pdg[i]:3d} ({mdst.mc_score_x[i]:7.1f},{mdst.mc_score_y[i]:7.1f}) pz={mdst.mc_score_pz[i]:7.5f}")

        print(f"Pz sum = {pzsum}")

        print("Score Clusters:")
    cl_idx = R.EAC.get_score_cluster_indexes(mdst.mc_score_pz, mdst.mc_score_x, mdst.mc_score_y, mdst.mc_score_z,
                                             mdst.ecal_cluster_x, mdst.ecal_cluster_y)
    x_aves = R.EAC.get_score_cluster_loc(cl_idx, mdst.mc_score_x, mdst.mc_score_pz)
    y_aves = R.EAC.get_score_cluster_loc(cl_idx, mdst.mc_score_y, mdst.mc_score_pz)
    pz_sums = R.EAC.get_score_cluster_pz(cl_idx, mdst.mc_score_pz)
    e_sums = R.EAC.get_score_cluster_e(cl_idx, mdst.mc_score_px, mdst.mc_score_py, mdst.mc_score_pz)
    # for i in range(len(cl_idx)):
    #     x_ave = 0
    #     y_ave = 0
    #     pzsum = 0
    #     for j in range(len(cl_idx[i])):
    #         idx = cl_idx[i][j]
    #         x_ave += mdst.mc_score_x[idx]*mdst.mc_score_pz[idx]
    #         y_ave += mdst.mc_score_y[idx]*mdst.mc_score_pz[idx]
    #         pzsum += mdst.mc_score_pz[idx]
    #
    #     x_ave = x_ave/pzsum
    #     y_ave = y_ave/pzsum
    for i in range(len(x_aves)):
        x_ave = x_aves[i]
        y_ave = y_aves[i]
        pzsum = pz_sums[i]
        if debug:
            print(f"i: {i:2d} N={len(cl_idx[i]):3d} ({R.EAC.ecal_xpos_to_index(x_ave):6.2f},{R.EAC.ecal_ypos_to_index(y_ave):6.2f}) ({x_ave:7.1f},{y_ave:7.1f}) pz = {pzsum:7.4f}")
        x_dots3.append(R.EAC.ecal_xpos_to_index(x_ave))
        y_dots3.append(R.EAC.ecal_ypos_to_index(y_ave))

    # Fill the individual score plane hits
    for i in range(len(mdst.mc_score_x)):
        if mdst.mc_score_pz[i] > 0.05 and mdst.mc_score_z[i] > 1440:
             x_dots4.append(R.EAC.ecal_xpos_to_index(mdst.mc_score_x[i]))
             y_dots4.append(R.EAC.ecal_ypos_to_index(mdst.mc_score_y[i]))

    x_clus_dot = array('d')
    y_clus_dot = array('d')
    x_clus_uncor_dot = array('d')
    y_clus_uncor_dot = array('d')

    for i in range(len(mdst.ecal_cluster_seed_ix)):
        if debug:
            print(f"Cluster: ({mdst.ecal_cluster_seed_ix[i]:3d},{mdst.ecal_cluster_seed_iy[i]:2d}) ({mdst.ecal_cluster_x[i]:6.2f},{mdst.ecal_cluster_y[i]:6.2f}) ({mdst.ecal_cluster_x[i]:6.2f}, {mdst.ecal_cluster_y[i]:6.2f}) E={mdst.ecal_cluster_energy[i]:7.4f} GeV")
        
        x_clus_dot.append(R.EAC.ecal_xpos_to_index(R.EAC.ecal_xpos_correction(mdst.ecal_cluster_x[i],mdst.ecal_cluster_energy[i],)))
        y_clus_dot.append(R.EAC.ecal_ypos_to_index(mdst.ecal_cluster_y[i]))
        x_clus_uncor_dot.append(R.EAC.ecal_xpos_to_index(mdst.ecal_cluster_x[i]))
        y_clus_uncor_dot.append(R.EAC.ecal_ypos_to_index(mdst.ecal_cluster_y[i]))


    x_track = array('d')
    y_track = array('d')
    for i in range(len(mdst.track_x_at_ecal)):
        if mdst.track_type[i] == 1:
            if debug:
                print(f"Track:            ({mdst.track_x_at_ecal[i]:6.2f},{mdst.track_y_at_ecal[i]:6.2f},{mdst.track_z_at_ecal[i]:6.2f}) P=({mdst.track_px[i]:6.2f},{mdst.track_py[i]:6.2f},{mdst.track_pz[i]:6.2f})")
            x_track.append(R.EAC.ecal_xpos_to_index(mdst.track_x_at_ecal[i]))
            y_track.append(R.EAC.ecal_ypos_to_index(mdst.track_y_at_ecal[i]))

    legend2 = R.TLegend(0.,0.85,0.25,0.99)

    if len(x_dots)>0:
        dot_graph = R.TGraph(len(x_dots), x_dots, y_dots)
        dot_graph.SetMarkerColor(R.kRed)
        dot_graph.SetMarkerSize(1.5)
        dot_graph.SetMarkerStyle(R.kFullCircle)
        dot_graph.Draw("P")
        legend2.AddEntry(dot_graph,"Score plane primary hit")

    if len(x_dots3)>0:
        dot3_graph = R.TGraph(len(x_dots3), x_dots3, y_dots3)
        dot3_graph.SetMarkerColor(R.kOrange+2)
        dot3_graph.SetMarkerSize(1.1)
        dot3_graph.SetMarkerStyle(R.kFullCircle)
        dot3_graph.Draw("P")
        legend2.AddEntry(dot3_graph,"Score plane clusters")

    if len(x_dots2)>0:
        dot2_graph = R.TGraph(len(x_dots2), x_dots2, y_dots2)
        dot2_graph.SetMarkerColor(R.kOrange)
        dot2_graph.SetMarkerSize(0.5)
        dot2_graph.SetMarkerStyle(R.kFullCircle)
        dot2_graph.Draw("P")
        legend2.AddEntry(dot2_graph,"Score plane secondary hit")

    if len(x_dots4)>0:
        #print("Plot the ecal hits")
        dot4_graph = R.TGraph(len(x_dots4), x_dots4, y_dots4)
        dot4_graph.SetMarkerColor(R.kRed)
        dot4_graph.SetMarkerSize(0.3)
        dot4_graph.SetMarkerStyle(R.kFullCircle)
        dot4_graph.Draw("P")
        legend2.AddEntry(dot4_graph,"Score hits")


    if len(x_clus_dot)>0:
        clus_dot_graph = R.TGraph(len(x_clus_dot), x_clus_dot, y_clus_dot)
        clus_dot_graph.SetMarkerColor(R.kGreen+1)
        clus_dot_graph.SetMarkerSize(1.)
        clus_dot_graph.SetMarkerStyle(R.kFullCircle)
        clus_dot_graph.Draw("P")
        legend2.AddEntry(clus_dot_graph,"Cluster center")

    if len(x_clus_uncor_dot)>0:
        clus_dot_uncor_graph = R.TGraph(len(x_clus_uncor_dot), x_clus_uncor_dot, y_clus_uncor_dot)
        clus_dot_uncor_graph.SetMarkerColor(R.kCyan)
        clus_dot_uncor_graph.SetMarkerSize(0.9)
        clus_dot_uncor_graph.SetMarkerStyle(R.kFullCircle)
        clus_dot_uncor_graph.Draw("P")
        legend2.AddEntry(clus_dot_uncor_graph,"Cluster center uncor")

    if len(x_track)>0:
        track_dot_graph = R.TGraph(len(x_track), x_track, y_track)
        track_dot_graph.SetMarkerColor(R.kViolet)
        track_dot_graph.SetMarkerSize(.5)
        track_dot_graph.SetMarkerStyle(R.kFullCircle)
        track_dot_graph.Draw("P")
        legend2.AddEntry(track_dot_graph,"Track pos at ecal")

    legend2.Draw()
    cc2.Draw()

In [8]:
mdst.Print()

======== Event: 0 Run: 0  ========
 Particles: 


In [9]:
primary_index = -1
def Print_Event():
    global event, primary_index

    print(f"event = {event} Run: {mdst.run_number}, Event Num:{mdst.event_number}")
    print("")
    print("Generator MC Particles:")
    Esum=0
    P_sum = [0,0,0]
    for i in primary_mc_part_idx:
        if mdst.mc_part_pdg[i] != 623:
            Esum += mdst.mc_part_energy[i]
            P_sum[0] += mdst.mc_part_px[i]
            P_sum[1] += mdst.mc_part_py[i]
            P_sum[2] += mdst.mc_part_pz[i]
            print(f"[{i:2d}] ({mdst.mc_part_pdg[i]:3d}) E={mdst.mc_part_energy[i]:6.3f} GeV p=({mdst.mc_part_px[i]:6.3f},{mdst.mc_part_py[i]:6.3f},{mdst.mc_part_pz[i]:6.3f}) ")
            
    print(f"Sums: E = {Esum:6.3f}  P = ({P_sum[0]:6.3f},{P_sum[1]:6.3f},{P_sum[2]:6.3f})")

    print("")
    print(f"ECal Clusters: N={len(mdst.ecal_cluster_seed_ix)} [seed index](x,y) E:")
    Ecal_clus_e_sum=0
    for i in range(len(mdst.ecal_cluster_seed_index)):
        Ecal_clus_e_sum += mdst.ecal_cluster_energy[i]
        print(f" [{mdst.ecal_cluster_seed_index[i]:3d}] ({mdst.ecal_cluster_x[i]:7.2f},{mdst.ecal_cluster_y[i]:7.2f}, E = {mdst.ecal_cluster_energy[i]:7.3f})")
    print(f"Ecal clusters energy sum = {Ecal_clus_e_sum:7.3f}")
    print("\n")
    print("ECal Hits:")
    print("   i |   x |   y |  Energy  ")
    esum=0
    print("----------------------------")
    for i in range(len(mdst.ecal_hit_index_x)):
        print(f" {i:3d} | {mdst.ecal_hit_index_x[i]:3d} | {mdst.ecal_hit_index_y[i]:3d} | {mdst.ecal_hit_energy[i]:7.5f} ")
        esum+=mdst.ecal_hit_energy[i]
    print(f"ECal hits energy sum = {esum}\n")

    track_momentum_sum = 0.
    for i in range(len(mdst.track_type)):
        if mdst.track_type[i]==1:  # Only look at the KF tracks
            p = np.sqrt(mdst.track_px[i]*mdst.track_px[i]+mdst.track_py[i]*mdst.track_py[i]+mdst.track_pz[i]*mdst.track_pz[i])
            track_momentum_sum += p
            print(f"Track p=({mdst.track_px[i]:5.2f},{mdst.track_py[i]:5.2f},{mdst.track_pz[i]:5.2f}) p={p:6.3f} "
                  f"at ecal=({mdst.track_x_at_ecal[i]:6.1f},{mdst.track_y_at_ecal[i]:6.1f},{mdst.track_z_at_ecal[i]:6.1f})")
    print(f"Track |p| sum = {track_momentum_sum:6.3f}")
    print()
    print("MC particle tree:")
    print_mc_particle_tree(mdst)
    print("")


In [12]:
cluster_ncontrib = []
cluster_centers = []
cluster_ave_x = []
cluster_ave_y = []
cluster_sum_pz = []

while True:
    event+=1
    ch.GetEntry(event)

    secondary_hits_energy = R.EAC.get_score_secondary_hits_energy(mdst.mc_part_sim_status, mdst.mc_score_part_idx, mdst.mc_score_z, mdst.mc_score_px, mdst.mc_score_py, mdst.mc_score_pz)
    primary_hits_energy = R.EAC.get_score_primary_hits_energy(mdst.mc_part_sim_status, mdst.mc_score_part_idx, mdst.mc_score_z, mdst.mc_score_px, mdst.mc_score_py, mdst.mc_score_pz)

    mc_score_cluster_indexes = R.EAC.get_score_cluster_indexes(mdst.mc_score_pz,
                                                               mdst.mc_score_x, mdst.mc_score_y, mdst.mc_score_z,
                                                               mdst.ecal_cluster_x, mdst.ecal_cluster_y)
    if len(mdst.ecal_cluster_energy) >= 1:
        break

#    if len(mc_score_cluster_indexes) >= 3:
#        break
#    if len(secondary_hits_energy) > 20:
#        break
#    if count>2:
#        break
#     if len(mdst.ecal_cluster_seed_ix) >= 3:
#         break
#     if len(mdst.ecal_cluster_seed_ix) >= 1:
#         break
   # break
   #  if len(primary_hits_energy)>0 and primary_hits_energy[0]>3.2:
   #      print(f"event = {event}  E = {primary_hits_energy[0]}")
   #      break
    # if len(primary_hits_energy)>0 and primary_hits_energy[0]>3.1:
    #     print(f"event = {event}  E = {primary_hits_energy[0]}")
    #     break

    if event%1000 == 0:
        print(f"event = {event:6d}")
    if event >= ch.GetEntries():
        break

primary_mc_part_idx = R.EAC.get_list_of_primary_mc(mdst.mc_part_sim_status)

Print_Event()
Show_Event()

event = 2 Run: 14700, Event Num:-7

Generator MC Particles:
[ 8] (-11) E= 1.239 GeV p=( 0.037,-0.061, 1.237) 
[13] ( 11) E= 0.182 GeV p=( 0.015, 0.001, 0.182) 
[19] ( 11) E= 2.318 GeV p=( 0.065, 0.003, 2.317) 
Sums: E =  3.740  P = ( 0.116,-0.057, 3.736)

ECal Clusters: N=2 [seed index](x,y) E:
 [  3] ( 206.23, -71.24, E =   0.707)
 [  8] ( 289.77, -72.52, E =   0.648)
Ecal clusters energy sum =   1.355


ECal Hits:
   i |   x |   y |  Energy  
----------------------------
   0 |  19 |  -4 | 0.04646 
   1 | -19 |  -4 | 0.00354 
   2 | -19 |  -4 | 0.00688 
   3 |  12 |  -4 | 0.32620 
   4 |  17 |  -3 | 0.01068 
   5 | -12 |  -1 | 0.00619 
   6 |  -6 |   5 | 0.00372 
   7 |  13 |  -4 | 0.14595 
   8 |  18 |  -4 | 0.30787 
   9 |  12 |  -5 | 0.06393 
  10 |  12 |  -3 | 0.02243 
  11 |  12 |  -3 | -0.00157 
  12 |  18 |  -5 | 0.09859 
  13 |  18 |   5 | 0.01206 
  14 |  19 |  -3 | 0.02744 
  15 |  19 |  -5 | 0.02035 
  16 |  19 |  -5 | 0.00306 
  17 | -12 |   2 | 0.00160 
  18 |   4 |   3 

TypeError: double Ecal_Analysis_Class::ecal_xpos_correction(double xpos, double energy, int pdg) =>
    TypeError: takes at least 3 arguments (2 given)

In [13]:
mdst.Print()
#cc2.SaveAs(f"Tritrig_event_{mdst.event_number}.pdf")

======== Event: -7 Run: 14700  ========
 Particles: 
   Particle 0: type: 0 pdg: -11 energy: 0 px: 0.0467398 py: -0.0682415 pz: 1.38347 track: 0 ecal_cluster: -1
    |- Track: type: 1 n_hits: 11 d0: -0.389125 phi0: 0.0337716 omega: -0.000186166 tan_lambda: -0.0492982 z0: -0.0246303 chi2: 15.2541 time: -32.7064 px: 0.0467398 py: -0.0682415 pz: 1.38347
     |- At ECal:  x: 226.662 y: -73.8669 z: 1448
   Particle 1: type: 0 pdg: 22 energy: 0.707036 px: 0.0994597 py: -0.0343587 pz: 0.699162 track: -1 ecal_cluster: 0
    |- Ecal Cluster: energy: 0.707036 time: 27.7406 x: 206.232 y: -71.2434 z: 1449.73 seed_ix: 12 seed_iy: -4 seed_energy: 0.326203
   Particle 2: type: 0 pdg: 22 energy: 0.647923 px: 0.126788 py: -0.0317312 pz: 0.634604 track: -1 ecal_cluster: 1
    |- Ecal Cluster: energy: 0.647923 time: 27.627 x: 289.772 y: -72.5212 z: 1450.38 seed_ix: 18 seed_iy: -4 seed_energy: 0.307869


In [21]:
i=1
print(mdst.ecal_cluster_x[i],mdst.ecal_cluster_uncor_x[i],R.EAC.ecal_xpos_correction(mdst.ecal_cluster_uncor_x[i],mdst.ecal_cluster_energy[i],11))

289.7720031738281 285.2875061035156 295.359073926081


In [15]:
mdst.ecal_cluster_uncor_x

vector<double>{ 203.73500, 285.28751 }